In [1]:
from kaggle_secrets import UserSecretsClient
import os

user_secrets = UserSecretsClient()
my_token = user_secrets.get_secret("GITHUB_TOKEN")

!git clone https://{my_token}@github.com/WebCrawlerr/GAT.git

Cloning into 'GAT'...
remote: Enumerating objects: 292, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 292 (delta 9), reused 10 (delta 3), pack-reused 271 (from 1)
Receiving objects: 100% (292/292), 791.28 KiB | 11.99 MiB/s, done.
Resolving deltas: 100% (175/175), done.


In [2]:
# Pobranie zmian
!git pull origin main
# Wyświetla listę plików w pobranym folderze
!ls -R  GAT

fatal: not a git repository (or any parent up to mount point /kaggle)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
GAT:
filter_protein.py  prepare_bindingdb_data.py  run_experiments.py
inspect_data.py    prepare_leash_data.py      src
leash_head.csv	   __pycache__		      tests
main.py		   README.md		      verify_changes.py
models		   requirements.txt

GAT/models:
 best_model.pth   fold_0_best_model.pth  'gat-inz (2).ipynb'

GAT/__pycache__:
filter_protein.cpython-312.pyc	prepare_leash_data.cpython-312.pyc

GAT/src:
config.py	    features.py   model.py     recover_results.py
data_processing.py  inference.py  optimize.py  train.py
dataset.py	    loss.py	  __pycache__  utils.py

GAT/src/__pycache__:
config.cpython-312.pyc		 model.cpython-312.pyc
data_processing.cpython-312.pyc  optimize.cpython-312.pyc
dataset.cpython-312.pyc		 train.cpython-312.pyc
features.cpython-312.pyc	 utils.cpython-312.pyc
loss.cpython-312.pyc

GAT/tests:
test_cv.py	  test_optimize.py 

In [3]:
import shutil
import os
import sys
# 1. Zmiana katalogu roboczego
os.chdir('./GAT')
print(f"Aktualny katalog: {os.getcwd()}")

Aktualny katalog: /kaggle/working/GAT


In [4]:
# Najpierw sprawdźmy wersję po restarcie (czasem GPU ma starszego PyTorcha)
import torch
version = torch.__version__
print(f"PyTorch version: {version}")

# Instalacja PyG i bibliotek pomocniczych - korzystamy z gotowych paczek .whl
#Ta komenda dynamicznie dobiera link do wersji PyTorcha
torch_version_suffix = version.split('+')[0].replace('.', '') # np. 260 lub 240
cuda_version_suffix = 'cu' + torch.version.cuda.replace('.', '') # np. cu121

print(f"Instaluję dla: torch-{torch_version_suffix} + {cuda_version_suffix}")

# Instalacja zależności
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv \
  -f https://data.pyg.org/whl/torch-{version}.html

# Instalacja reszty z pliku (bez ponownego instalowania torch-geometric)
!pip install -r requirements.txt

PyTorch version: 2.6.0+cu124
Instaluję dla: torch-260 + cu124
Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 131.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 110.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:

In [5]:
import torch
import torch_geometric

print(f"CUDA dostępne: {torch.cuda.is_available()}")
print(f"Wersja PyTorch: {torch.__version__}")
print(f"Wersja PyG: {torch_geometric.__version__}")

# Próba importu kluczowych warstw
try:
    from torch_geometric.nn import GATConv
    print("✅ GATConv zaimportowany pomyślnie!")
except ImportError as e:
    print(f"❌ Błąd importu GATConv: {e}")

# Sprawdzenie czy mamy backendy (scatter/sparse)
# PyG 2.7+ potrafi czasem działać bez nich (wolniej), ale warto sprawdzić
try:
    import torch_scatter
    print("✅ torch_scatter obecny")
except ImportError:
    print("⚠️ torch_scatter BRAK (Może to spowolnić trening lub wywołać błąd)")

try:
    import torch_sparse
    print("✅ torch_sparse obecny")
except ImportError:
    print("⚠️ torch_sparse BRAK (Wymagany dla niektórych operatorów)")

CUDA dostępne: True
Wersja PyTorch: 2.6.0+cu124
Wersja PyG: 2.7.0
✅ GATConv zaimportowany pomyślnie!
✅ torch_scatter obecny
✅ torch_sparse obecny


In [6]:
#!python main.py --raw_file /kaggle/input/bindingdb-smiles/BindingDB_All.tsv --processed_dir ./data/processed --cv 5

In [7]:
#!python main.py --optimize --n_trials 15 --raw_file /kaggle/input/bindingdb-smiles/BindingDB_All.tsv --processed_dir ./data/processed
#--cv 5

In [8]:
#!python run_experiments.py --processed_dir ./data/processed --raw_file /kaggle/input/bindingdb-smiles/BindingDB_All.tsv

In [9]:
#PRZYGOTOWANIE DANYCH BELKA
#!python prepare_leash_data.py --input_file /kaggle/input/leash-BELKA/train.csv --output_dir /kaggle/working/ --protein BRD4 --negative_rate 1

In [10]:
#trening na przefiltrowanych belka z notatnika
#!python main.py --filtered_file /kaggle/input/gat-inz/leash_brd4_filtered.csv

In [11]:
#!python main.py \
#--filtered_file /kaggle/input/gat-inz/leash_brd4_filtered.csv \
#--processed_dir . \
#--limit 2500000 \
#--cv 0

In [12]:
!python -m src.inference \
  --model_path /kaggle/input/nabu-1/pytorch/default/1/best_model_1point8mil_data.pth \
  --test_file /kaggle/input/leash-BELKA/test.csv \
  --batch_size 4096 \
  --output_file submission.csv

Using device: cuda
Loading test data from /kaggle/input/leash-BELKA/test.csv...
Processing...
Sampling from /kaggle/input/leash-BELKA/test.csv using Polars (Test Mode=True)...
Processing test data (preserving all rows, ignoring labels)...
Final Test Dataset: 1674896 samples
Done!
Dataset loaded. Size: 1674896. Columns: ['id', 'buildingblock1_smiles', 'buildingblock2_smiles', 'buildingblock3_smiles', 'Ligand SMILES', 'protein_name']
Initializing model (Features: Node=68, Edge=6)...
Loading checkpoint: /kaggle/input/nabu-1/pytorch/default/1/best_model_1point8mil_data.pth
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/kaggle/working/GAT/src/inference.py", line 113, in <module>
    run_inference(args.model_path, args.test_file, args.output_file, args.batch_size, args.limit)
  File "/kaggle/working/GAT/src/inference.py", line 47, in run_inference
    model.load_state_dict(torch.load(model_p

In [13]:
#!python src/recover_results.py